<a href="https://colab.research.google.com/github/devinbook/Tagalog-profanity-detection/blob/main/Data_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers torch datasets scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import Dataset

In [ ]:
df = pd.read_csv('/content/tagalog_profanity_feature_extracted (1).csv')

In [ ]:
dataset = Dataset.from_pandas(df)

In [ ]:
from transformers import DistilBertTokenizer

# Initialize tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    examples['Text'] = [str(text) for text in examples['Text']]
    tokens = tokenizer(examples['Text'], padding='max_length', truncation=True)
    tokens['labels'] = examples['Label']  # Reference the correct label column
    return tokens

# Apply the tokenization function
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Tokenize the entire dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/36784 [00:00<?, ? examples/s]

Map:   0%|          | 0/36784 [00:00<?, ? examples/s]

In [ ]:
train_test_split = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

In [ ]:
from transformers import DistilBertForSequenceClassification

# Load DistilBERT model for sequence classification
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,  # Train for 5 epochs
    weight_decay=0.01
)

# Define the compute metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    acc = accuracy_score(labels, predictions)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.107800,0.075195,0.978116,0.978287,0.979212,0.977365
2,0.056400,0.067391,0.984640,0.984732,0.987534,0.981946
3,0.031200,0.065977,0.986408,0.986512,0.987578,0.985449


TrainOutput(global_step=5520, training_loss=0.08926289890879306, metrics={'train_runtime': 4405.4523, 'train_samples_per_second': 20.039, 'train_steps_per_second': 1.253, 'total_flos': 1.1694354420750336e+16, 'train_loss': 0.08926289890879306, 'epoch': 3.0})

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'df' is a pandas DataFrame
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)  # 70% train, 30% temp
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)  # 15% val, 15% test

# Check the sizes
print(f"Train size: {len(train_df)}, Validation size: {len(val_df)}, Test size: {len(test_df)}")

Train size: 25748, Validation size: 5518, Test size: 5518


In [ ]:
# Tokenize the test set
test_encodings = tokenizer(test_df['Text'].tolist(), truncation=True, padding=True, return_tensors="pt")


In [ ]:
class TestDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

# Create test dataset
test_labels = test_df['Label'].tolist()
test_dataset = TestDataset(test_encodings, test_labels)

In [ ]:
!pip install datasets transformers

import datasets
from transformers import Trainer

In [ ]:
test_dataset_hf = datasets.Dataset.from_dict(test_encodings)
test_dataset_hf = test_dataset_hf.add_column("labels", test_labels)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
)

test_results = trainer.evaluate(test_dataset_hf)
print(test_results)

{'eval_loss': 0.02230621874332428, 'eval_model_preparation_time': 0.0028, 'eval_accuracy': 0.9945632475534614, 'eval_f1': 0.9946217282179993, 'eval_precision': 0.9928418038654259, 'eval_recall': 0.9964080459770115, 'eval_runtime': 66.873, 'eval_samples_per_second': 82.515, 'eval_steps_per_second': 5.159}


In [ ]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Assuming `model` is your trained DistilBERT model
model_save_path = '/content/drive/MyDrive/Saved Models'  # Specify your desired save path

# Save the model and tokenizer
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

print(f"Model and tokenizer saved to {model_save_path}")

Model and tokenizer saved to /content/drive/MyDrive/Saved Models
